<a href="https://colab.research.google.com/github/andmartins7/plagio-lexico-e-semantico/blob/main/plagio_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import numpy as np
nltk.download('punkt')

In [ ]:
# Texto Original - Base
texto_original = """
A administração é uma arte e ciência vital para o sucesso e sustentabilidade de qualquer organização. Ao explorar os conceitos fundamentais de administração e organização, entendemos que a administração é o processo de planejar, organizar, liderar e controlar recursos para alcançar objetivos definidos, enquanto a organização é um conjunto de pessoas trabalhando juntas em uma estrutura formal para alcançar metas comuns.

No contexto brasileiro, a administração enfrenta desafios específicos, como a alta carga tributária, os elevados custos de financiamento, a burocracia exacerbada, a instabilidade política e econômica, a desigualdade social e as deficiências de infraestrutura.
"""

In [ ]:
# Geração de Exemplos de Plágio Léxico
def gerar_plagio_lexico(texto, n=3):
    frases = nltk.sent_tokenize(texto, language="portuguese")
    exemplos = []
    for _ in range(n):
        exemplo = random.sample(frases, len(frases))  # Rearranjo das frases
        exemplos.append(" ".join(exemplo))
    return exemplos

plagios_lexicos = gerar_plagio_lexico(texto_original)
print("Exemplos de Plágio Léxico:")
for exemplo in plagios_lexicos:
    print(exemplo)
    print("-"*80)

In [ ]:
# Geração de Exemplos de Plágio Semântico com Transformers (Paráfrases)
paraphrase_pipeline = pipeline("summarization", model="tuner007/pegasus_paraphrase")

def gerar_plagio_semantico(texto, n=3):
    frases = nltk.sent_tokenize(texto, language="portuguese")
    exemplos = []
    for frase in frases:
        # Limitar o tamanho da frase para evitar problemas de comprimento
        if len(frase.split()) > 50:
            frase = " ".join(frase.split()[:50])
        paraphrases = paraphrase_pipeline(frase, num_return_sequences=n, max_length=60, truncation=True)
        exemplos.extend([p['summary_text'] for p in paraphrases])
    return exemplos

plagios_semanticos = gerar_plagio_semantico(texto_original, n=1)
print("\nExemplos de Plágio Semântico:")
for exemplo in plagios_semanticos:
    print(exemplo)
    print("-"*80)

In [ ]:
# Criação do Vetor TF-IDF para Detecção de Similaridade Léxica
todos_textos = [texto_original] + plagios_lexicos + plagios_semanticos
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(todos_textos)

# Similaridade de Coseno
def detectar_plagio_lexico(tfidf_matrix, idx_base=0):
    similaridades = cosine_similarity(tfidf_matrix[idx_base], tfidf_matrix)
    return similaridades

similaridades = detectar_plagio_lexico(X)
print("\nMatriz de Similaridade (Léxico):")
print(similaridades)

In [ ]:
# Avaliação do Modelo - Similaridade Semântica
# Comparação dos textos originais vs parafraseados utilizando embeddings de transformers
similarity_pipeline = pipeline("feature-extraction", model="sentence-transformers/paraphrase-MiniLM-L6-v2")

def calcular_similaridade_semantica(texto1, texto2):
    embedding1 = np.mean(similarity_pipeline(texto1)[0], axis=0)
    embedding2 = np.mean(similarity_pipeline(texto2)[0], axis=0)
    similarity = cosine_similarity([embedding1], [embedding2])
    return similarity[0][0]

for idx, texto in enumerate(plagios_semanticos):
    similaridade = calcular_similaridade_semantica(texto_original, texto)
    print(f"Similaridade Semântica do Texto Original com Plágio Semântico {idx+1}: {similaridade}")

In [ ]:
# Resultados e Conclusões
print("\nA matriz de similaridade e os valores de similaridade semântica indicam quais textos foram plagiados e com qual intensidade, permitindo uma análise clara de plágio léxico e semântico.")